In [119]:
using Pkg
Pkg.activate("../")
using HTTP
using JSON
using JSON3
using Dates
using OrderedCollections
include("./DIVAndFairEase.jl")
token = ENV["beaconAPItoken"];

  Activating project at `~/Projects/DIVAnd-FAIR-EASE`


In [120]:
datadir = "../data/"
footprintdir = "../Footprint"
mkpath.([datadir, footprintdir])

2-element Vector{String}:
 "../data"
 "../Footprint"

In [121]:
const beacon_services = OrderedDict(
    "Euro-Argo" => "https://beacon-argo.maris.nl",
    "CORA Profile" => "https://beacon-cora-pr.maris.nl",
    "CORA Timeseries" => "https://beacon-cora-ts.maris.nl",
    "EMODnet Chemistry" => "https://beacon-emod-chem.maris.nl",
    "World Ocean Database" => "https://beacon-wod.maris.nl",
    "SeaDataNet CDI TS" => "https://beacon-cdi-ts.maris.nl",
    "CMEMS BGC" => "https://beacon-cmems.maris.nl",
);

In [140]:
datasource = "CORA Timeseries"
domain = [12., 18., 43., 46.]
parameter1 = "TEMP"
if datasource == "World Ocean Database"
    parameter1 = "Temperature"
elseif datasource == "SeaDataNet CDI TS"
    parameter1 = "ITS-90 water temperature"
end
parameter2 = "TEMP"
datestart = Dates.Date(2010, 1, 1)
dateend = Dates.Date(2015, 1, 1)
minlon = domain[1]
maxlon = domain[2]
minlat = domain[3]
maxlat = domain[4]
mindepth = 0.
maxdepth = 10.
dateref = Dates.Date(1950, 1, 1) #Days since 1950
mintemporal = (datestart - dateref).value #Days since 1950
maxtemporal = (dateend - dateref).value #Days since 1950
regionname = "BlackSea"

@info(datestart, dateend)

┌ Info: 2010-01-01
└   dateend = 2015-01-01


In [141]:
beaconURL = beacon_services[datasource]
footprintURL = joinpath(beaconURL, "api/datasets/footprint")
datasource_name = replace(datasource, " " => "_")
footprintfile = joinpath(footprintdir, "Footprint_$(datasource_name).json")
@info("Footprint endpoint: $(footprintURL)")
@info("Footprint file: $(footprintfile)")

[ Info: Footprint endpoint: https://beacon-cora-ts.maris.nl/api/datasets/footprint
[ Info: Footprint file: ../Footprint/Footprint_CORA_Timeseries.json


In [142]:
if isfile(footprintfile)
    @info("Footprint file already downloaded")
else
    @info("Writing Footprint file")
    open(footprintfile, "w") do io
        r = HTTP.request("GET", footprintURL, 
            ["Authorization" => "Bearer $(token)"],
            response_stream=io)
        @info(r.status)
    end
end

[ Info: Footprint file already downloaded


In [143]:
# Note 1: the JSON file is 193M, just to get the list of variables?

In [144]:
data = JSON.parsefile(footprintfile);
attributes = data["unique_column_attributes"]
params = sort(collect(keys(attributes)));
parameter1 in params

true

In [145]:
query = DIVAndFairEase.prepare_query_new(datasource, parameter1, parameter2, datestart, dateend, 
    mindepth, maxdepth, minlon, maxlon, minlat, maxlat)

[ Info: Working with CORA dataset
[ Info: Working with CORA dataset


"{\"query_parameters\":[{\"column_name\":\"TEMP\",\"alias\":\"TEMP\"},{\"column_name\":\"JULD\",\"alias\":\"TIME\"},{\"column_name\":\"DEPH\",\"alias\":\"DEPTH\"},{\"column_name\":\"LONGITUDE\",\"alias\":\"LONGITUDE\"},{\"column_name\":\"LATITUDE\",\"alias\":\"LATITUDE\"}],\"filters\":[{\"for_query_parameter\":\"TIME\",\"min\":21915,\"max\":23741},{\"for_query_parameter\":\"DEPTH\",\"min\":0.0,\"max\":10.0},{\"for_query_parameter\":\"LONGITUDE\",\"min\":12.0,\"max\":18.0},{\"for_query_parameter\":\"LATITUDE\",\"min\":43.0,\"max\":46.0},{\"for_query_parameter\":\"TEMP\",\"min\":-2.0,\"max\":30.0}],\"output\":{\"format\":\"netcdf\"}}"

In [146]:
# Note 2: cannot work with such a long list of parameters!!

In [147]:
filename = joinpath(datadir, "$(datasource_name)_$(replace(parameter1, " "=>"_"))_$(regionname)_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.nc");
filename

"../data/CORA_Timeseries_TEMP_BlackSea_20100101-20150101_0-10m.nc"

In [153]:
@info("Data will be written in file:\n$(filename)")

if isfile(filename)
    @info("File already downloaded")
    rm(filename)
end
@time open(filename, "w") do io
    r = HTTP.request("POST", joinpath(beaconURL, "api/query"), 
        ["Content-Type" => "application/json",
         "Authorization" => "Bearer $(token)"
        ],
        query, 
        response_stream=io);
    @info(r.status)
end

@info("NetCDF file size: $(round(filesize(filename)/1000^2, digits=1))M")

┌ Info: Data will be written in file:
└ ../data/CORA_Timeseries_TEMP_BlackSea_20100101-20150101_0-10m.nc
[ Info: File already downloaded


LoadError: HTTP.Exceptions.StatusError(400, "POST", "/api/query", HTTP.Messages.Response:
"""
HTTP/1.1 400 Bad Request
Content-Type: application/json
Vary: Access-Control-Request-Headers
x-beacon-query-id: c4ef5eb1-2a3a-431f-8f65-ca57e8a80ff8
X-Powered-By: ARR/3.0
Date: Fri, 06 Sep 2024 14:29:30 GMT
Content-Length: 94

[Message Body was streamed]""")

In [149]:
r = HTTP.request("POST", joinpath(beaconURL, "api/query"), 
    ["Authorization" => "Bearer $(token)", "Content-Type" => "application/"],
    query)
@info(r.status)

[ Info: 200


In [150]:
r.body

UInt8[]

In [132]:
query

"{\"query_parameters\":[{\"column_name\":\"TEMP\",\"alias\":\"TEMP\"},{\"column_name\":\"JULD\",\"alias\":\"TIME\"},{\"column_name\":\"DEPH\",\"alias\":\"DEPTH\"},{\"column_name\":\"LONGITUDE\",\"alias\":\"LONGITUDE\"},{\"column_name\":\"LATITUDE\",\"alias\":\"LATITUDE\"}],\"filters\":[{\"for_query_parameter\":\"TIME\",\"min\":23376,\"max\":23741},{\"for_query_parameter\":\"DEPTH\",\"min\":0.0,\"max\":10.0},{\"for_query_parameter\":\"LONGITUDE\",\"min\":12.0,\"max\":18.0},{\"for_query_parameter\":\"LATITUDE\",\"min\":43.0,\"max\":46.0},{\"for_query_parameter\":\"TEMP\",\"min\":-2.0,\"max\":30.0}],\"output\":{\"format\":\"netcdf\"}}"